In [3]:
#Importing the necessary libraries
from bs4 import BeautifulSoup
from decimal import Decimal
import pandas as pd
import numpy as np
import os
import re

In [332]:
#Main Code:

#Define the directory where the Silkroad2 Folders are stored
directory = r"C:\Users\nshah\Desktop\silkroad2"

for roots1, dirs1, files1 in os.walk(directory):
    for subdir1 in dirs1:
        if "2013-" in subdir1 or "2014-" in subdir1:
            items_textlist = []
            users_textlist = []
            extradets1 = []
            extradets2 = []
            vendorlist = []
            datelist = []
            datelist2 = []
            categ_textlist = []
            errorlist1 = []
            errorlist2 = []
            errorlist3 = []
            takedate = subdir1
            datedirectory = os.path.join(roots1,subdir1)
            for roots2, dirs2, files2 in os.walk(datedirectory):
                for subdir2 in dirs2:
                    
                    #Loop for subsection "items"
                    if subdir2 == "items":
                        itemdirectory = os.chdir(os.path.join(roots2,subdir2))
                        for itemfile in os.listdir(itemdirectory):
                            extradets=[]
                            with open(itemfile,'rb') as html_file:
                                errorlist1.append(itemfile)
                                content = html_file.read()
                                soup = BeautifulSoup(content, 'lxml')
                                paratag = soup.find_all('p')
                                tag = soup.find_all('h3')
                                tagtextlist = []
                                for p in paratag:
                                    paratext = p.text
                                    tagtextlist.append(paratext)
                                for h3 in tag:
                                    tagtext = h3.text
                                    tagtextlist.append(tagtext)
                                try:
                                    for elem in tagtextlist:
                                        if "vendor:" in elem:
                                            vendorname = elem
                                    vendorname = vendorname.split(':')[1].strip()
                                except IndexError:
                                    vendorname = "None"    
                                    
                                if "no feedback" in tagtextlist:
                                    itemtextlistbuffer = ["No Feedback","No Feedback","No Feedback","No Feedback"]
                                    extradetsbuffer = ["A","B",itemfile,takedate]
                                    vendorbuffer = ["No Feedback","No Feedback","No Feedback",vendorname]
                                    items_textlist.extend(itemtextlistbuffer)
                                    extradets.extend(extradetsbuffer)
                                    vendorlist.extend(vendorbuffer)
                                    extradets2 = extradets
                                    
                                elif "item feedback" in tagtextlist:
                                    try:
                                        table_itemfeedback = soup.find_all('tbody')[2]
                                        tds = table_itemfeedback.find_all('td')
                                        for td in tds:
                                            text = td.text
                                            text = text.strip()
                                            items_textlist.append(text)
                                            extradets.append(itemfile)
                                            extradets.append(takedate)
                                            vendorlist.append(vendorname)
                                        extradets1 = extradets[:len(extradets)//2]
                                        extradets2.extend(extradets1)
                                    except IndexError:
                                        pass
                                            
                                elif "vendor feedback" in tagtextlist:
                                    try:                                        
                                        table_itemfeedback = soup.find_all('tbody')[1]
                                        tds = table_itemfeedback.find_all('td')
                                        for td in tds:
                                            text = td.text
                                            text = text.strip()
                                            items_textlist.append(text)
                                            extradets.append(itemfile)
                                            extradets.append(takedate)
                                            vendorlist.append(vendorname)
                                        extradets1 = extradets[:len(extradets)//2]
                                        extradets2.extend(extradets1)
                                    except IndexError:
                                        pass
                            
                            if len(items_textlist) %4 != 0:
                                errorlist2.append(itemfile)
                                j=0
                                spacechecklist=[]
                                spacechecklist=items_textlist[-2:]
                                for i in spacechecklist:
                                    if i=='':
                                        j+=1
                                if j != 0:
                                    items_textlist = items_textlist[:-j]
                                    vendorlist = vendorlist[:-j]
                                    extradets = extradets[:-(j*2)]
                                    extradets2 = extradets2[:-j]
                                missing = len(items_textlist) % 4
                                newitemtextlist = ["NA"]
                                newextradetslist = [itemfile,takedate]
                                newvendornamelist = [vendorname]
                                if missing == 1:
                                    extradets2 = []
                                    items_textlist.extend(newitemtextlist*3)
                                    extradets.extend(newextradetslist*3)
                                    extradets1 = extradets[:len(extradets)//2]
                                    extradets2.extend(extradets1)
                                    vendorlist.extend(newvendornamelist*3)
                                if missing == 2:
                                    items_textlist.extend(newitemtextlist*missing)
                                    extradets2.extend(newextradetslist)
                                    vendorlist.extend(newvendornamelist*missing)
                                if missing == 3:
                                    extradets2 = []
                                    items_textlist.extend(newitemtextlist)
                                    extradets.extend(newextradetslist)
                                    extradets1 = extradets[:len(extradets)//2]
                                    extradets2.extend(extradets1)
                                    vendorlist.extend(newvendornamelist)
                            errorlist3.append(len(extradets2))
                        
                        #Generating the dataframe under "items" subcategory for the entire folder 
                        df1 = pd.DataFrame(np.array(items_textlist).reshape(-1,4), columns = ["Rating","Feedback","Item","Freshness"])
                        df_vendorfeedback = df1
                        df_vendorfeedback.drop("Item", axis=1, inplace=True)
                        dfitemname = pd.DataFrame(np.array(extradets2).reshape(-1,4), columns = ["A","B","ItemName","Date"])
                        dfitemname.drop(["A","B"], axis=1, inplace=True)
                        df_vendorfeedback[["Item_Name","IssueDate"]] = dfitemname[["ItemName","Date"]]
                        dfvendorname = pd.DataFrame(np.array(vendorlist).reshape(-1,4), columns = ["A","B","C","Vendor"])
                        dfvendorname.drop(["A","B","C"], axis=1, inplace=True)
                        df_vendorfeedback["Vendor_Name"] = dfvendorname["Vendor"]

                        
                    #Loop for subsection "users"    
                    if subdir2 == "users":
                        userdirectory = os.path.join(roots2,subdir2)
                        for roots3, dirs3, files3 in os.walk(userdirectory):
                            for subfiles3 in files3:
                                indfilesdirectory = os.path.join(roots3,subfiles3)
                                with open(indfilesdirectory,'rb') as html_file:
                                    try:
                                        content = html_file.read()
                                        soup = BeautifulSoup(content, 'lxml')
                                        listings = soup.find_all('div', class_ = 'item')
                                        listings2 = listings[1:]
                                        for listing in listings2:
                                            item = listing.find('div', class_ = 'item_title').text
                                            detail = listing.find('div', class_ = 'item_details').text.replace(' ','').strip()
                                            detailsplit = detail.split() 
                                            if "vendor:" in ''.join(detailsplit):
                                                for i in detailsplit:
                                                    detail2 = i.split(':')[1]
                                                    users_textlist.append(detail2)
                                                users_textlist.append(item)
                                                datelist2.append(takedate)
                                                try:
                                                    price = listing.find('div', class_ = 'price_big').text
                                                    users_textlist.append(price)
                                                except AttributeError:
                                                    pass
                                            else:
                                                soldby = listing.find('div', class_ = 'vendor').text.strip()
                                                soldbysplit = soldby.split('by ')
                                                soldbyname = soldbysplit[1].split()
                                                soldbyname = soldbyname[:-1]
                                                if len(soldbyname) > 1:
                                                    vendor_name = ' '.join(soldbyname)
                                                    users_textlist.append(vendor_name)
                                                else:
                                                    try:
                                                        vendor_name = soldbyname[0]
                                                        users_textlist.append(vendor_name)
                                                    except IndexError:
                                                        vendor_name = "<missing>"
                                                        users_textlist.append(vendor_name)
                                                pricebtc = detailsplit[0]
                                                detailsplit=detailsplit[1:]
                                                for i in detailsplit:                                                          
                                                    detail2 = i.split(':')[1]
                                                    users_textlist.append(detail2)
                                                users_textlist.append(item)
                                                users_textlist.append(pricebtc)
                                                datelist2.append(takedate)
                                    except AttributeError:
                                        pass

                        #Generating the dataframe under "users" subcategory for the entire folder             
                        df2 = pd.DataFrame(np.array(users_textlist).reshape(-1,5), columns = ["Seller","ShipFrom","ShipTo","Item","Price(in BTC)"])
                        df_userlistings = df2
                        df_userlistings['Price(in BTC)'] = df_userlistings['Price(in BTC)'].astype(str).str.extract('(\d+\.?\d*)')
                        df_userlistings['Price(in BTC)'] = df_userlistings['Price(in BTC)'].astype(float)
                        df_datelist = pd.DataFrame(datelist, columns = ["Date"])
                        df_userlistings["Issue_Date"] = df_datelist["Date"]
                        df_userlistings = df_userlistings[["Item","Seller","Price(in BTC)","ShipFrom","ShipTo","Issue_Date"]]

                        
                    #Loop for subsection "categories"    
                    if subdir2 == "categories":
                        categdirectory = os.path.join(roots2,subdir2)            
                        for roots4, dirs4, files4 in os.walk(categdirectory):
                            for subdir4 in dirs4:
                                if "cocaine" in subdir4:
                                    cocainedirectory = os.chdir(os.path.join(roots4,subdir4))
                                    for indfileofcocaine in os.listdir(cocainedirectory):
                                        with open(indfileofcocaine,'rb') as html_file:
                                            content = html_file.read()
                                            soup = BeautifulSoup(content, 'lxml')
                                            cocaines = soup.find_all('div', class_ = 'item')
                                            cocaines2 = cocaines[1:]
                                            for cocaine in cocaines2:
                                                item = cocaine.find('div', class_ = 'item_title').text
                                                detail = cocaine.find('div', class_ = 'item_details').text.replace(' ','').strip()
                                                detailsplit = detail.split()
                                                if "vendor:" in ''.join(detailsplit):
                                                    for i in detailsplit:
                                                        detail2 = i.split(':')[1]
                                                        categ_textlist.append(detail2)
                                                    price = cocaine.find('div', class_ = 'price_big').text
                                                    categ_textlist.append(item)
                                                    categ_textlist.append(price)
                                                    datelist2.append(takedate)

                                                else:
                                                    soldby = cocaine.find('div', class_ = 'vendor').text.strip()
                                                    soldbysplit = soldby.split('by ')
                                                    soldbyname = soldbysplit[1].split()
                                                    soldbyname = soldbyname[:-1]
                                                    if len(soldbyname) > 1:
                                                        vendor_name = ' '.join(soldbyname)
                                                        categ_textlist.append(vendor_name)
                                                    else:
                                                        try:
                                                            vendor_name = soldbyname[0]
                                                            categ_textlist.append(vendor_name)
                                                        except IndexError:
                                                            vendor_name = "<missing>"
                                                            categ_textlist.append(vendor_name)
                                                    pricebtc = detailsplit[0]
                                                    detailsplit=detailsplit[1:]
                                                    for i in detailsplit:                                                          
                                                        detail2 = i.split(':')[1]
                                                        categ_textlist.append(detail2)
                                                    categ_textlist.append(item)
                                                    categ_textlist.append(pricebtc)
                                                    datelist2.append(takedate)
                        
                        #Generating the dataframe under "categories" subcategory for the entire folder 
                        df3 = pd.DataFrame(np.array(categ_textlist).reshape(-1,5), columns = ["Seller","ShipFrom","ShipTo","Item","Price(in BTC)"])
                        df_categories = df3
                        df_categories['Price(in BTC)'] = df_categories['Price(in BTC)'].astype(str).str.extract('(\d+\.?\d*)')
                        df_categories['Price(in BTC)'] = df_categories['Price(in BTC)'].astype(float)
                        df_datelist2 = pd.DataFrame(datelist2, columns = ["Date"])
                        df_categories["Issue_Date"] = df_datelist2["Date"]
                        df_categories = df_categories[["Item","Seller","Price(in BTC)","ShipFrom","ShipTo","Issue_Date"]]
            
            
            # Writing all three dataframes into an excel sheet named by the individual folder name
            
            if len(df_vendorfeedback) > 1048576:
                df_vendorfeedback1sthalf = df_vendorfeedback[0:980004]
                df_vendorfeedback2ndhalf = df_vendorfeedback[980004:]
                writer = pd.ExcelWriter(f"C:\\Users\\nshah\\Desktop\\AllFiles\\{takedate}.xlsx", engine='xlsxwriter')
                df_vendorfeedback1sthalf.to_excel(writer, sheet_name='VendorFeedbackSheet1', index = False)
                df_vendorfeedback2ndhalf.to_excel(writer, sheet_name='VendorFeedbackSheet2', index = False)
                df_userlistings.to_excel(writer, sheet_name='IndividualUserListings', index = False)
                df_categories.to_excel(writer, sheet_name='Categories', index = False)
                writer.save()
            
            elif len(df_userlistings) > 1048576:
                df_userlistings1sthalf = df_usrelistings[0:980004]
                df_userlistings2ndhalf = df_userlistings[980004:]
                writer = pd.ExcelWriter(f"C:\\Users\\nshah\\Desktop\\AllFiles\\{takedate}.xlsx", engine='xlsxwriter')
                df_vendorfeedback.to_excel(writer, sheet_name='VendorFeedback', index = False)
                df_userlistings1sthalf.to_excel(writer, sheet_name='IndividualUserListingSheet1', index = False)
                df_userlistings2ndhalf.to_excel(writer, sheet_name='IndividualUserListingSheet2', index = False)
                df_categories.to_excel(writer, sheet_name='Categories', index = False)
                writer.save()
                
            else:
                writer = pd.ExcelWriter(f"C:\\Users\\nshah\\Desktop\\AllFiles\\{takedate}.xlsx", engine='xlsxwriter')
                df_vendorfeedback.to_excel(writer, sheet_name='VendorFeedback', index = False)
                df_userlistings.to_excel(writer, sheet_name='IndividualUserListings', index = False)
                df_categories.to_excel(writer, sheet_name='Categories', index = False)
                writer.save()